# Bug triage with Deep Learning - DWEN

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
best_loss = float('inf')
best_epoch = 0
verbose = 0
loss = 0

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'openoffice'
METHOD = 'baseline_dwen_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


72234

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 5.45 s, sys: 620 ms, total: 6.06 s
Wall time: 5.98 s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [13]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids


In [14]:
baseline.train_data[:10]

[[59, 27],
 [59, 92],
 [27, 92],
 [64, 43],
 [44, 45],
 [53, 54],
 [84, 63],
 [75, 699],
 [105, 121],
 [186, 199]]

In [15]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [16]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '4\n',
 'bug_status': '1\n',
 'component': '18\n',
 'creation_ts': '2002-08-31 16:50:00 +0000',
 'delta_ts': '2013-02-24 20:34:09 +0000',
 'description': '[CLS] it \' s not same format in the di ##c files o ##o make self as in swedish di ##c file i can download . when i register and add the file in o ##o i can \' t see any words . when i look in the map there is 2 files with al ##mo ##ast same name . se _ se ##1 . di ##c is made by o ##o self and contain no words . i try ##ed to convert th file to same format as so ##ffi ##ce . di ##c with lit ##te or no progress . the new file can i import but it stops in the middle of " a " i can \' t do anymore to solve the problem so i leave it to yu ##o guys . regards ana ##kins [SEP]',
 'description_bert': '[CLS] it \' s not same format in the di ##c files o ##o make self as in swedish di ##c file i can download . when i register and add the file in o ##o i can \' t see any words . when i look in the map there is 2 files with al 

### Generating the batch test

In [17]:
"Train ", len(baseline.dup_sets_train)

('Train ', 11043)

In [18]:
import random
random.choice(list(issues_by_buckets))

93976

In [19]:
110647 in experiment.baseline.bug_set

True

In [20]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator(None, 
                                                                                          baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1,
                                                                                          issues_by_buckets)

valid_title_sample_a = np.concatenate([valid_input_sample['title'], valid_input_sample['title']], 0)
valid_title_sample_b = np.concatenate([valid_input_pos['title'], valid_input_neg['title']], 0)
valid_desc_sample_a = np.concatenate([valid_input_sample['description'], valid_input_sample['description']], 0)
valid_desc_sample_b = np.concatenate([valid_input_pos['description'], valid_input_neg['description']], 0)

test_gen = ([valid_title_sample_a, valid_title_sample_b, valid_desc_sample_a, valid_desc_sample_b], valid_sim)

# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 29.6 ms, sys: 4.08 ms, total: 33.6 ms
Wall time: 33.3 ms


In [21]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_sim.shape

((128, 20), (128, 20), (128,))

### Validar entrada

In [22]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [23]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [24]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 18562'

In [25]:
# import io

# def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
#     embeddings_index = {}
#     embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
#     f = open(embed_path, 'rb')
#     f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, f.readline().split())

#     vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
#     vocab_size = len(vocab) 

#     # Initialize uniform the vector considering the Tanh activation
#     embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
#     embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

#     loop = tqdm(f)
#     loop.set_description("Loading FastText")
#     for line in loop:
#         tokens = line.rstrip().split(' ')
#         embed = list(map(float, tokens[1:]))
#         word = tokens[0]
#         embeddings_index[word] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#     f.close()
#     loop.close()

#     print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

#     loop = tqdm(total=vocab_size)
#     loop.set_description('Loading embedding from dataset pretrained')
#     i = 0
#     for word, embed in vocab.items():
#         if word in embeddings_index:
#             embedding_matrix[i] = embeddings_index[word]
#         else:
#             embedding_matrix[i] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#         i+=1
#     loop.close()
#     baseline.embedding_matrix = embedding_matrix

In [26]:
def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [27]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 22s, sys: 3.1 s, total: 1min 25s
Wall time: 1min 24s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [28]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [29]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### DWEN model

In [30]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, \
    Average, GlobalAveragePooling1D, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
import keras.backend as K

def dwen_feature(title_feature_model, desc_feature_model, \
                  sequence_length_t, sequence_length_d, name):
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    
    # Embedding feature
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    
    bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    bug_d_feat = GlobalAveragePooling1D()(bug_d_feat)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = Average(name = 'merge_features_{}'.format(name))([bug_t_feat, bug_d_feat])
    
    bug_feature_model = Model(inputs=[bug_t, bug_d], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

def dwen_model(bug_feature_output_a, bug_feature_output_b, name):
    
    inputs = np.concatenate([bug_feature_output_a.input, bug_feature_output_b.input], -1).tolist()
    
    bug_feature_output_a = bug_feature_output_a.output
    bug_feature_output_b = bug_feature_output_b.output
    
    # 2D concatenate feature
    bug_feature_output = concatenate([bug_feature_output_a, bug_feature_output_b])
    
    hidden_layers = 2
    
    # Deep Hidden MLPs
    for _ in range(hidden_layers):
        number_of_units = K.int_shape(bug_feature_output)[1]
        bug_feature_output = Dense(number_of_units // 2)(bug_feature_output)
#         bug_feature_output = BatchNormalization()(bug_feature_output)
        bug_feature_output = Activation('relu')(bug_feature_output)
        #bug_feature_output = Dropout(.5)(bug_feature_output)
    
     # Sigmoid
    output = Dense(1, activation='sigmoid')(bug_feature_output)

    similarity_model = Model(inputs=inputs, outputs=[output], name = 'dwen_output')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    return similarity_model

In [31]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Similarity model
bug_feature_output_a = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_a')
bug_feature_output_b = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_b')
similarity_model = dwen_model(bug_feature_output_a, bug_feature_output_b, 'dwen')

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(None, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                        bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0],  h[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}".format(epoch+1, h[0],  h[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_

Epoch: 120 Loss: 0.65, acc: 0.61
Epoch: 121 Loss: 0.68, acc: 0.53
Epoch: 122 Loss: 0.62, acc: 0.66
Epoch: 123 Loss: 0.67, acc: 0.59
Epoch: 124 Loss: 0.64, acc: 0.65
Epoch: 125 Loss: 0.61, acc: 0.66
Epoch: 126 Loss: 0.61, acc: 0.69
Epoch: 127 Loss: 0.64, acc: 0.60
Epoch: 128 Loss: 0.60, acc: 0.71
Epoch: 129 Loss: 0.66, acc: 0.62
Epoch: 130 Loss: 0.65, acc: 0.63
Epoch: 131 Loss: 0.66, acc: 0.62
Epoch: 132 Loss: 0.64, acc: 0.65
Epoch: 133 Loss: 0.63, acc: 0.66
Epoch: 134 Loss: 0.62, acc: 0.64
Epoch: 135 Loss: 0.68, acc: 0.60
Epoch: 136 Loss: 0.64, acc: 0.59
Epoch: 137 Loss: 0.66, acc: 0.66
Epoch: 138 Loss: 0.71, acc: 0.54
Epoch: 139 Loss: 0.65, acc: 0.60
Epoch: 140 Loss: 0.63, acc: 0.66
Epoch: 141 Loss: 0.64, acc: 0.58
Epoch: 142 Loss: 0.61, acc: 0.65
Epoch: 143 Loss: 0.62, acc: 0.70
Epoch: 144 Loss: 0.66, acc: 0.62
Epoch: 145 Loss: 0.67, acc: 0.62
Epoch: 146 Loss: 0.65, acc: 0.63
Epoch: 147 Loss: 0.66, acc: 0.63
Epoch: 148 Loss: 0.65, acc: 0.63
Epoch: 149 Loss: 0.67, acc: 0.54
Epoch: 150

Epoch: 372 Loss: 0.60, acc: 0.69
Epoch: 373 Loss: 0.66, acc: 0.60
Epoch: 374 Loss: 0.62, acc: 0.63
Epoch: 375 Loss: 0.62, acc: 0.66
Epoch: 376 Loss: 0.62, acc: 0.65
Epoch: 377 Loss: 0.62, acc: 0.66
Epoch: 378 Loss: 0.64, acc: 0.62
Epoch: 379 Loss: 0.61, acc: 0.70
Epoch: 380 Loss: 0.63, acc: 0.62
Epoch: 381 Loss: 0.59, acc: 0.66
Epoch: 382 Loss: 0.63, acc: 0.63
Epoch: 383 Loss: 0.60, acc: 0.70
Epoch: 384 Loss: 0.58, acc: 0.70
Epoch: 385 Loss: 0.61, acc: 0.62
Epoch: 386 Loss: 0.67, acc: 0.59
Epoch: 387 Loss: 0.70, acc: 0.53
Epoch: 388 Loss: 0.61, acc: 0.65
Epoch: 389 Loss: 0.62, acc: 0.69
Epoch: 390 Loss: 0.59, acc: 0.68
Epoch: 391 Loss: 0.63, acc: 0.61
Epoch: 392 Loss: 0.56, acc: 0.73
Epoch: 393 Loss: 0.62, acc: 0.62
Epoch: 394 Loss: 0.60, acc: 0.65
Epoch: 395 Loss: 0.61, acc: 0.70
Epoch: 396 Loss: 0.65, acc: 0.64
Epoch: 397 Loss: 0.60, acc: 0.67
Epoch: 398 Loss: 0.61, acc: 0.70
Epoch: 399 Loss: 0.64, acc: 0.66
Epoch: 400 Loss: 0.61, acc: 0.63
Epoch: 401 Loss: 0.60, acc: 0.69
Epoch: 402

Epoch: 624 Loss: 0.54, acc: 0.74
Epoch: 625 Loss: 0.60, acc: 0.73
Epoch: 626 Loss: 0.59, acc: 0.70
Epoch: 627 Loss: 0.59, acc: 0.66
Epoch: 628 Loss: 0.65, acc: 0.66
Epoch: 629 Loss: 0.67, acc: 0.59
Epoch: 630 Loss: 0.62, acc: 0.64
Epoch: 631 Loss: 0.61, acc: 0.68
Epoch: 632 Loss: 0.60, acc: 0.66
Epoch: 633 Loss: 0.62, acc: 0.68
Epoch: 634 Loss: 0.60, acc: 0.73
Epoch: 635 Loss: 0.60, acc: 0.71
Epoch: 636 Loss: 0.56, acc: 0.72
Epoch: 637 Loss: 0.60, acc: 0.66
Epoch: 638 Loss: 0.61, acc: 0.62
Epoch: 639 Loss: 0.58, acc: 0.67
Epoch: 640 Loss: 0.62, acc: 0.68
Epoch: 641 Loss: 0.55, acc: 0.68
Epoch: 642 Loss: 0.60, acc: 0.68
Epoch: 643 Loss: 0.61, acc: 0.66
Epoch: 644 Loss: 0.61, acc: 0.66
Epoch: 645 Loss: 0.65, acc: 0.62
Epoch: 646 Loss: 0.54, acc: 0.74
Epoch: 647 Loss: 0.57, acc: 0.69
Epoch: 648 Loss: 0.55, acc: 0.74
Epoch: 649 Loss: 0.66, acc: 0.65
Epoch: 650 Loss: 0.56, acc: 0.73
Epoch: 651 Loss: 0.68, acc: 0.62
Epoch: 652 Loss: 0.60, acc: 0.65
Epoch: 653 Loss: 0.60, acc: 0.67
Epoch: 654

Epoch: 876 Loss: 0.63, acc: 0.68
Epoch: 877 Loss: 0.57, acc: 0.71
Epoch: 878 Loss: 0.57, acc: 0.72
Epoch: 879 Loss: 0.62, acc: 0.62
Epoch: 880 Loss: 0.60, acc: 0.68
Epoch: 881 Loss: 0.62, acc: 0.62
Epoch: 882 Loss: 0.61, acc: 0.72
Epoch: 883 Loss: 0.61, acc: 0.66
Epoch: 884 Loss: 0.66, acc: 0.61
Epoch: 885 Loss: 0.53, acc: 0.69
Epoch: 886 Loss: 0.58, acc: 0.71
Epoch: 887 Loss: 0.58, acc: 0.70
Epoch: 888 Loss: 0.60, acc: 0.68
Epoch: 889 Loss: 0.62, acc: 0.68
Epoch: 890 Loss: 0.62, acc: 0.64
Epoch: 891 Loss: 0.62, acc: 0.67
Epoch: 892 Loss: 0.64, acc: 0.66
Epoch: 893 Loss: 0.61, acc: 0.65
Epoch: 894 Loss: 0.59, acc: 0.72
Epoch: 895 Loss: 0.62, acc: 0.68
Epoch: 896 Loss: 0.55, acc: 0.68
Epoch: 897 Loss: 0.66, acc: 0.60
Epoch: 898 Loss: 0.60, acc: 0.67
Epoch: 899 Loss: 0.64, acc: 0.65
Epoch: 900 Loss: 0.64, acc: 0.64
Epoch: 901 Loss: 0.57, acc: 0.73
Epoch: 902 Loss: 0.60, acc: 0.67
Epoch: 903 Loss: 0.61, acc: 0.67
Epoch: 904 Loss: 0.62, acc: 0.65
Epoch: 905 Loss: 0.60, acc: 0.66
Epoch: 906

In [32]:
_[:20]

['108544:111059,109674,108379,109366|89620:0.638342946767807,96654:0.6307685673236847,91612:0.6286369562149048,96181:0.6194678246974945,73946:0.6190367937088013,98890:0.6184544265270233,116193:0.6168799698352814,117315:0.6158325374126434,99100:0.6154980063438416,94181:0.6152200698852539,95068:0.6129328310489655,94909:0.6125856041908264,112672:0.6117276847362518,101671:0.6112997829914093,107139:0.6094796657562256,99874:0.6094290912151337,122052:0.6088610589504242,97773:0.6086090505123138,92051:0.6063841581344604,103396:0.6059651672840118,91855:0.6050929725170135,28932:0.6042809188365936,98451:0.6040443181991577,118218:0.6036084592342377,102225:0.6026510894298553,109950:0.5995483100414276,85170:0.5985702872276306,55499:0.5979336798191071,96299:0.5974744558334351',
 '109674:108544,111059,108379,109366|99541:0.41072118282318115,93934:0.3501514196395874,100293:0.34763187170028687,100449:0.3395318388938904,105082:0.3392093777656555,119486:0.33525556325912476,123048:0.32127201557159424,99451:

In [33]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                            bug_train_ids, method='dwen')

"recall@25 last epoch:", recall

('recall@25 last epoch:', 0.23)

### Retrieval evaluation

In [34]:
print("Total of queries:", len(retrieval.test))

Total of queries: 2086


#### Getting the model trained

In [35]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'baseline_dwen_1000_feature_1000epochs_64batch(openoffice)'

In [36]:
model = bug_feature_output_a
#model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [37]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_layer_title (Embeddin (None, 20, 300)      5568600     title_dwen_a[0][0]               
__________________________________________________________________________________________________
embedding_layer_desc (Embedding (None, 20, 300)      5568600     desc_dwen_a[0][0]                
__________________________________________________________________________________________________
global_ave

In [38]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='dwen')

In [39]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/exported_rank_baseline_dwen_1000.txt'

In [40]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [41]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.14,
 '2 - recall_at_10': 0.16,
 '3 - recall_at_15': 0.19,
 '4 - recall_at_20': 0.22,
 '5 - recall_at_25': 0.23}